In [1]:
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings
import time
import os

CHROMA_PATH = "chroma_db_test_21"

PROMPT_TEMPLATE = """
Answer the question based on the following context and your knowledge:

{context}

---

Answer the question based on the above context and your knowledge: {question} """

In [2]:


def get_embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(prompt)

    model = Ollama(model="llama3.2:1b")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    print(sources)
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text


In [3]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Début du traitement")
embedding_function = get_embedding_function()
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
#retriever = db.as_retriever()
print("HHHHHHH")
try:
    results = db.similarity_search_with_score("XML", k=9)

except Exception as e:
    print(f"Erreur rencontrée : {e}")
print(results)



INFO:__main__:Début du traitement
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


HHHHHHH
[(Document(metadata={'id': 'D:\\Generative IA\\rag_pdfs\\pdf_data\\PowerStudio_DDE_XML_en.pdf:2:1', 'page': 2, 'source': 'D:\\Generative IA\\rag_pdfs\\pdf_data\\PowerStudio_DDE_XML_en.pdf'}, page_content='Area Network including the Internet; the only requirement is that  it has an IP address to be questioned. In this way, any \nelectrical or physical parameter may be requested which, using a peripheral, is collected by the PowerStudio System & \nPowerStudio Scada. In this way, using the XML communication pr otocol, many parameters can be collected from one or more \nPowerStudios installed on the same network including accessible external networks. \n \nThe sentences that must be made for th e XML protocol and for the DDE server, must be correctly formed. If not, the \napplication will return a variable error. \n  \nThe chain “some.xml ”, refers to instant variable requests: \n \nIn the following example, an XML request is shown requesti ng the L1 phase-neutral voltage from the 

In [4]:
query_rag("c'est qoui le power studio scada de luxe et power studio en terme de cout performance (la difference thecnique) ")

Human: 
Answer the question based on the following context and your knowledge:

PowerStudio  
 
 52                                                                                                                                  User Manual  
 
6.2.3  Editing Tools  
 
Operation of positioning tools is the same as that explained in Chapter 5.2.6 but applied to 
SCADA screen controls.

---

PowerStudio  
 
 44                                                                                                                                  User Manual  
 
The 
 Save option will allow the changes made on the screen to be saved and the 
 Save as  
option will enable a copy of the screen to be saved with a different name.  
Finally, ‘Options’  will allow the dimensions of the scr een selected at the time to be changed.  
 
 
The width and height of the screen is measured in pixels and the size of the grid will specify, in 
pixels, the distance between a point on the grid and another in horizo

'En fonction du contenu fourni, il semble que la réponse soit :\n\nPowerStudio SCADA de luxe.\n\nPuisque l\'article mentionne les fonctionnalités "Save", "Save as" et "Options" pour sauvegarder les modifications apportées à la fenêtre SCADA, cela suggère que le PowerStudio SCADA de luxe est conçu pour gérer les couches de performance.'

In [6]:
#pip install langchain chromadb huggingface_hub
